In [ ]:
## Objetivo: Avaliar o comportamento de clientes de um E-commerce de Suplementos Alimentares

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
from datetime import datetime, timedelta

In [143]:
df = pd.read_excel("data.xlsx")

In [145]:
df

,user_id,purchase_date
0,1,2023-01-01
1,1,2023-02-01
2,1,2023-12-01
3,2,2023-01-01
4,2,2023-06-01
...,...,...
241,229,2023-10-01
242,229,2023-08-01
243,230,2023-09-01
244,230,2023-10-01


In [15]:
              
df['purchase_date'] = pd.to_datetime(df['purchase_date'])


In [69]:
first_purchase

,user_id,first_purchase_date
0,1,2023-01-01
1,2,2023-01-01
2,3,2023-01-01
3,4,2023-01-01
4,5,2023-02-01
...,...,...
120,227,2023-07-01
121,228,2023-07-01
122,229,2023-08-01
123,230,2023-09-01


In [17]:
# Obter a primeira compra de cada usuário agrupando pela última data
first_purchase = df.groupby('user_id')['purchase_date'].min().reset_index()
first_purchase.columns = ['user_id', 'first_purchase_date']

In [71]:
last_purchase

,user_id,last_purchase_date
0,1,2023-12-01
1,2,2023-12-01
2,3,2023-03-01
3,4,2023-04-01
4,5,2023-03-01
...,...,...
120,227,2023-07-01
121,228,2023-08-01
122,229,2023-10-01
123,230,2023-10-01


In [19]:
# Obter a última compra de cada usuário, agrupando as maiores datas da tabela
last_purchase = df.groupby('user_id')['purchase_date'].max().reset_index()
last_purchase.columns = ['user_id', 'last_purchase_date']

In [73]:
# Mesclar com o dataframe original para obter todas as compras com a data da primeira e ultima compra (Simular ao PROCV no Excel)

# Tabela original x first_purchase
df = df.merge(first_purchase, on='user_id')

# Tabela original x last_purchase
df = df.merge(last_purchase, on='user_id')

In [81]:
# Exibindo a tabela mesclada com as informações de primeiras e últimas compras de acordo com o user_id:

df

,user_id,purchase_date,first_purchase_date_x,last_purchase_date_x,first_purchase_month,purchase_month,is_new_customer,cliente_retido_30,cliente_reativado,days_since_last_purchase,cliente_risco_churn,cliente_churn,first_purchase_date_y,last_purchase_date_y
0,1,2023-01-01,2023-01-01,2023-12-01,2023-01,2023-01,True,True,False,31,False,False,2023-01-01,2023-12-01
1,1,2023-02-01,2023-01-01,2023-12-01,2023-01,2023-02,False,True,False,31,False,False,2023-01-01,2023-12-01
2,1,2023-12-01,2023-01-01,2023-12-01,2023-01,2023-12,False,True,False,31,False,False,2023-01-01,2023-12-01
3,2,2023-01-01,2023-01-01,2023-12-01,2023-01,2023-01,True,False,True,31,True,False,2023-01-01,2023-12-01
4,2,2023-06-01,2023-01-01,2023-12-01,2023-01,2023-06,False,False,True,31,True,False,2023-01-01,2023-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,229,2023-10-01,2023-08-01,2023-10-01,2023-08,2023-10,False,True,False,92,True,False,2023-08-01,2023-10-01
242,229,2023-08-01,2023-08-01,2023-10-01,2023-08,2023-08,True,True,False,92,True,False,2023-08-01,2023-10-01
243,230,2023-09-01,2023-09-01,2023-10-01,2023-09,2023-09,True,True,False,92,True,False,2023-09-01,2023-10-01
244,230,2023-10-01,2023-09-01,2023-10-01,2023-09,2023-10,False,True,False,92,True,False,2023-09-01,2023-10-01


In [23]:
# Identificar clientes novos (primeira compra no mês)
df['first_purchase_month'] = df['first_purchase_date'].dt.to_period('M')
df['purchase_month'] = df['purchase_date'].dt.to_period('M')
df['is_new_customer'] = df['first_purchase_month'] == df['purchase_month']

In [25]:
# ANALISE DE RETENÇÃO
#CLIENTES RETIDOS SÃO AQUELES CLIENTES QUE FIZERAM UMA NOVA COMPRA ANTES O PERÍODO DE 30 DIAS APÓS A PRIMEIRA COMPRA

In [27]:
# Adicionar uma coluna que verifica se houve uma compra dentro de 30 dias após a primeira compra
df['cliente_retido_aux'] = (df['purchase_date'] <= df['first_purchase_date'] + pd.Timedelta(days=31)) & (df['purchase_date'] > df['first_purchase_date'])

In [93]:
df

,user_id,purchase_date,first_purchase_date_x,last_purchase_date_x,first_purchase_month,purchase_month,is_new_customer,cliente_retido_30,cliente_reativado,days_since_last_purchase,cliente_risco_churn,cliente_churn,first_purchase_date_y,last_purchase_date_y,cliente_retido_aux
0,1,2023-01-01,2023-01-01,2023-12-01,2023-01,2023-01,True,True,False,31,False,False,2023-01-01,2023-12-01,True
1,1,2023-02-01,2023-01-01,2023-12-01,2023-01,2023-02,False,True,False,31,False,False,2023-01-01,2023-12-01,True
2,1,2023-12-01,2023-01-01,2023-12-01,2023-01,2023-12,False,True,False,31,False,False,2023-01-01,2023-12-01,True
3,2,2023-01-01,2023-01-01,2023-12-01,2023-01,2023-01,True,False,True,31,True,False,2023-01-01,2023-12-01,False
4,2,2023-06-01,2023-01-01,2023-12-01,2023-01,2023-06,False,False,True,31,True,False,2023-01-01,2023-12-01,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,229,2023-10-01,2023-08-01,2023-10-01,2023-08,2023-10,False,True,False,92,True,False,2023-08-01,2023-10-01,True
242,229,2023-08-01,2023-08-01,2023-10-01,2023-08,2023-08,True,True,False,92,True,False,2023-08-01,2023-10-01,True
243,230,2023-09-01,2023-09-01,2023-10-01,2023-09,2023-09,True,True,False,92,True,False,2023-09-01,2023-10-01,True
244,230,2023-10-01,2023-09-01,2023-10-01,2023-09,2023-10,False,True,False,92,True,False,2023-09-01,2023-10-01,True


In [29]:
# Agrupar por user_id para obter a informação de retenção apenas uma vez por usuário
df_retencao = df.groupby('user_id').agg({
    'cliente_retido_aux': 'max'
}).reset_index()

In [97]:
df_retencao

,user_id,cliente_retido_aux
0,1,True
1,2,False
2,3,True
3,4,True
4,5,True
...,...,...
120,227,False
121,228,True
122,229,True
123,230,True


In [83]:
# Juntar a classificação de retenção ao dataframe original (simular a um Left Join)

df = df.merge(df_retencao[['user_id', 'cliente_retido_aux']], on='user_id', how='left',  suffixes=('', '_'))

In [99]:
# Remover a coluna temporária do dataframe original
df.drop(columns=['cliente_retido_aux'], inplace=True)

# Renomear a coluna para cliente_retido_30
df.rename(columns={'cliente_retido_aux_': 'cliente_retido_30'}, inplace=True)

In [103]:
df

,user_id,purchase_date,first_purchase_date_x,last_purchase_date_x,first_purchase_month,purchase_month,is_new_customer,cliente_retido_30,cliente_reativado,days_since_last_purchase,cliente_risco_churn,cliente_churn,first_purchase_date_y,last_purchase_date_y
0,1,2023-01-01,2023-01-01,2023-12-01,2023-01,2023-01,True,True,False,31,False,False,2023-01-01,2023-12-01
1,1,2023-02-01,2023-01-01,2023-12-01,2023-01,2023-02,False,True,False,31,False,False,2023-01-01,2023-12-01
2,1,2023-12-01,2023-01-01,2023-12-01,2023-01,2023-12,False,True,False,31,False,False,2023-01-01,2023-12-01
3,2,2023-01-01,2023-01-01,2023-12-01,2023-01,2023-01,True,False,True,31,True,False,2023-01-01,2023-12-01
4,2,2023-06-01,2023-01-01,2023-12-01,2023-01,2023-06,False,False,True,31,True,False,2023-01-01,2023-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,229,2023-10-01,2023-08-01,2023-10-01,2023-08,2023-10,False,True,False,92,True,False,2023-08-01,2023-10-01
242,229,2023-08-01,2023-08-01,2023-10-01,2023-08,2023-08,True,True,False,92,True,False,2023-08-01,2023-10-01
243,230,2023-09-01,2023-09-01,2023-10-01,2023-09,2023-09,True,True,False,92,True,False,2023-09-01,2023-10-01
244,230,2023-10-01,2023-09-01,2023-10-01,2023-09,2023-10,False,True,False,92,True,False,2023-09-01,2023-10-01


In [35]:
# ANALISE DE REATIVAÇÃO
#CLIENTES REATIVADOS SÃO AQUELES CLIENTES QUE FIZERAM UMA NOVA COMPRA DEPOIS DO PERÍODO DE 30 DIAS (APÓS A PRIMEIRA COMPRA)


# Adicionar uma coluna que verifica se houve uma compra depois de 30 dias após a primeira compra e o cliente não é retido
df['cliente_reativado_aux'] = (df['purchase_date'] >= df['first_purchase_date'] + pd.Timedelta(days=31)) & (df['cliente_retido_30'] != True)

In [113]:
df

,user_id,purchase_date,first_purchase_date_x,last_purchase_date_x,first_purchase_month,purchase_month,is_new_customer,cliente_retido_30,cliente_reativado,days_since_last_purchase,cliente_risco_churn,cliente_churn,first_purchase_date_y,last_purchase_date_y
0,1,2023-01-01,2023-01-01,2023-12-01,2023-01,2023-01,True,True,False,31,False,False,2023-01-01,2023-12-01
1,1,2023-02-01,2023-01-01,2023-12-01,2023-01,2023-02,False,True,False,31,False,False,2023-01-01,2023-12-01
2,1,2023-12-01,2023-01-01,2023-12-01,2023-01,2023-12,False,True,False,31,False,False,2023-01-01,2023-12-01
3,2,2023-01-01,2023-01-01,2023-12-01,2023-01,2023-01,True,False,True,31,True,False,2023-01-01,2023-12-01
4,2,2023-06-01,2023-01-01,2023-12-01,2023-01,2023-06,False,False,True,31,True,False,2023-01-01,2023-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,229,2023-10-01,2023-08-01,2023-10-01,2023-08,2023-10,False,True,False,92,True,False,2023-08-01,2023-10-01
242,229,2023-08-01,2023-08-01,2023-10-01,2023-08,2023-08,True,True,False,92,True,False,2023-08-01,2023-10-01
243,230,2023-09-01,2023-09-01,2023-10-01,2023-09,2023-09,True,True,False,92,True,False,2023-09-01,2023-10-01
244,230,2023-10-01,2023-09-01,2023-10-01,2023-09,2023-10,False,True,False,92,True,False,2023-09-01,2023-10-01


In [37]:
# Agrupar por user_id para obter a informação de reativação apenas uma vez por usuário
df_reativacao= df.groupby('user_id').agg({
    'cliente_reativado_aux': 'max'
}).reset_index()

In [117]:
df_reativacao

,user_id,cliente_reativado_aux
0,1,False
1,2,True
2,3,False
3,4,False
4,5,False
...,...,...
120,227,False
121,228,False
122,229,False
123,230,False


In [39]:
# Juntar a classificação de retenção ao dataframe original
df = df.merge(df_reativacao[['user_id', 'cliente_reativado_aux']], on='user_id', how='left',  suffixes=('', '_'))# Remover a coluna temporária do dataframe original
df.drop(columns=['cliente_reativado_aux'], inplace=True)

df.rename(columns={'cliente_reativado_aux_': 'cliente_reativado'}, inplace=True)


In [123]:
df

,user_id,purchase_date,first_purchase_date_x,last_purchase_date_x,first_purchase_month,purchase_month,is_new_customer,cliente_retido_30,cliente_reativado,days_since_last_purchase,cliente_risco_churn,cliente_churn,first_purchase_date_y,last_purchase_date_y
0,1,2023-01-01,2023-01-01,2023-12-01,2023-01,2023-01,True,True,False,31,False,False,2023-01-01,2023-12-01
1,1,2023-02-01,2023-01-01,2023-12-01,2023-01,2023-02,False,True,False,31,False,False,2023-01-01,2023-12-01
2,1,2023-12-01,2023-01-01,2023-12-01,2023-01,2023-12,False,True,False,31,False,False,2023-01-01,2023-12-01
3,2,2023-01-01,2023-01-01,2023-12-01,2023-01,2023-01,True,False,True,31,True,False,2023-01-01,2023-12-01
4,2,2023-06-01,2023-01-01,2023-12-01,2023-01,2023-06,False,False,True,31,True,False,2023-01-01,2023-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,229,2023-10-01,2023-08-01,2023-10-01,2023-08,2023-10,False,True,False,92,True,False,2023-08-01,2023-10-01
242,229,2023-08-01,2023-08-01,2023-10-01,2023-08,2023-08,True,True,False,92,True,False,2023-08-01,2023-10-01
243,230,2023-09-01,2023-09-01,2023-10-01,2023-09,2023-09,True,True,False,92,True,False,2023-09-01,2023-10-01
244,230,2023-10-01,2023-09-01,2023-10-01,2023-09,2023-10,False,True,False,92,True,False,2023-09-01,2023-10-01


In [41]:
# ANALISE DE CHURN


#CLIENTES CLASSIFICADOS COMO RISCO_CHURN SÃO QUE FORAM RETIDOS OU REATIVADOS MAS NÃO FAZEM COMPRAS HÁ MAIS DE 90 DIAS

#verifica quantidade de dias entre a ultima compra
data_atual="01-01-2024"
data_atual=pd.to_datetime(data_atual)
df['days_since_last_purchase'] = (data_atual - df['last_purchase_date']).dt.days


In [125]:
df

,user_id,purchase_date,first_purchase_date_x,last_purchase_date_x,first_purchase_month,purchase_month,is_new_customer,cliente_retido_30,cliente_reativado,days_since_last_purchase,cliente_risco_churn,cliente_churn,first_purchase_date_y,last_purchase_date_y
0,1,2023-01-01,2023-01-01,2023-12-01,2023-01,2023-01,True,True,False,31,False,False,2023-01-01,2023-12-01
1,1,2023-02-01,2023-01-01,2023-12-01,2023-01,2023-02,False,True,False,31,False,False,2023-01-01,2023-12-01
2,1,2023-12-01,2023-01-01,2023-12-01,2023-01,2023-12,False,True,False,31,False,False,2023-01-01,2023-12-01
3,2,2023-01-01,2023-01-01,2023-12-01,2023-01,2023-01,True,False,True,31,True,False,2023-01-01,2023-12-01
4,2,2023-06-01,2023-01-01,2023-12-01,2023-01,2023-06,False,False,True,31,True,False,2023-01-01,2023-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,229,2023-10-01,2023-08-01,2023-10-01,2023-08,2023-10,False,True,False,92,True,False,2023-08-01,2023-10-01
242,229,2023-08-01,2023-08-01,2023-10-01,2023-08,2023-08,True,True,False,92,True,False,2023-08-01,2023-10-01
243,230,2023-09-01,2023-09-01,2023-10-01,2023-09,2023-09,True,True,False,92,True,False,2023-09-01,2023-10-01
244,230,2023-10-01,2023-09-01,2023-10-01,2023-09,2023-10,False,True,False,92,True,False,2023-09-01,2023-10-01


In [43]:
# Adicionar uma coluna que verifica se o cliente  foi retido ou reativado e sua ultima compra  faz mais de 90 dias
df['cliente_risco_churn'] = (df['cliente_reativado'] == True) | (df['cliente_retido_30'] == True) & (df['days_since_last_purchase'] > 90)


In [127]:
df

,user_id,purchase_date,first_purchase_date_x,last_purchase_date_x,first_purchase_month,purchase_month,is_new_customer,cliente_retido_30,cliente_reativado,days_since_last_purchase,cliente_risco_churn,cliente_churn,first_purchase_date_y,last_purchase_date_y
0,1,2023-01-01,2023-01-01,2023-12-01,2023-01,2023-01,True,True,False,31,False,False,2023-01-01,2023-12-01
1,1,2023-02-01,2023-01-01,2023-12-01,2023-01,2023-02,False,True,False,31,False,False,2023-01-01,2023-12-01
2,1,2023-12-01,2023-01-01,2023-12-01,2023-01,2023-12,False,True,False,31,False,False,2023-01-01,2023-12-01
3,2,2023-01-01,2023-01-01,2023-12-01,2023-01,2023-01,True,False,True,31,True,False,2023-01-01,2023-12-01
4,2,2023-06-01,2023-01-01,2023-12-01,2023-01,2023-06,False,False,True,31,True,False,2023-01-01,2023-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,229,2023-10-01,2023-08-01,2023-10-01,2023-08,2023-10,False,True,False,92,True,False,2023-08-01,2023-10-01
242,229,2023-08-01,2023-08-01,2023-10-01,2023-08,2023-08,True,True,False,92,True,False,2023-08-01,2023-10-01
243,230,2023-09-01,2023-09-01,2023-10-01,2023-09,2023-09,True,True,False,92,True,False,2023-09-01,2023-10-01
244,230,2023-10-01,2023-09-01,2023-10-01,2023-09,2023-10,False,True,False,92,True,False,2023-09-01,2023-10-01


In [45]:
#CLIENTES CLASSIFICADOS COMO CHURN SÃO QUE NÃO FORAM RETIDOS OU REATIVADOS E NÃO FAZEM COMPRAS HÁ MAIS DE 90 DIAS

# Adicionar uma coluna que verifica se o cliente não foi retido nem reativado e sua primeira compra faz mais de 90
df['cliente_churn'] = (df['cliente_reativado'] != True) & (df['cliente_retido_30'] != True) & (df['days_since_last_purchase'] > 90)

In [47]:
#Calcular a taxa de retenção
total_data = first_purchase.groupby(first_purchase['first_purchase_date'].dt.to_period('M')).agg({'user_id': 'nunique'})


retention_data = df[df['cliente_retido_30']].groupby(df['first_purchase_date'].dt.to_period('M')).agg({'user_id': 'nunique'})
retention_rate = (retention_data / total_data).fillna(0) * 100

In [129]:
retention_rate

,user_id
first_purchase_date,
2023-01,75.000000
2023-02,75.000000
2023-03,33.333333
2023-04,62.500000
2023-05,33.333333
2023-06,44.444444
2023-07,25.000000
2023-08,50.000000
2023-09,50.000000


In [49]:
#Calcular a taxa de churn
churn_data = df[df['cliente_churn']].groupby(df['first_purchase_date'].dt.to_period('M')).agg({'user_id': 'nunique'})
churn_rate = (churn_data / total_data).fillna(0) * 100

In [133]:
churn_rate

,user_id
first_purchase_date,
2023-01,0.000000
2023-02,12.500000
2023-03,16.666667
2023-04,37.500000
2023-05,46.666667
2023-06,55.555556
2023-07,56.250000
2023-08,41.666667
2023-09,50.000000


In [51]:
#Calcular a taxa de risco_churn
risk_churn_data = df[df['cliente_risco_churn']].groupby(df['first_purchase_date'].dt.to_period('M')).agg({'user_id': 'nunique'})
risk_churn_rate = (risk_churn_data / total_data).fillna(0) * 100

In [135]:
risk_churn_rate

,user_id
first_purchase_date,
2023-01,75.000000
2023-02,62.500000
2023-03,83.333333
2023-04,62.500000
2023-05,53.333333
2023-06,44.444444
2023-07,43.750000
2023-08,58.333333
2023-09,50.000000


In [53]:
#Calcular a taxa de reativação
revival_data = df[df['cliente_reativado']].groupby(df['first_purchase_date'].dt.to_period('M')).agg({'user_id': 'nunique'})
revival_rate = (revival_data / total_data).fillna(0) * 100

In [139]:
revival_rate

,user_id
first_purchase_date,
2023-01,25.000000
2023-02,12.500000
2023-03,50.000000
2023-04,0.000000
2023-05,20.000000
2023-06,0.000000
2023-07,18.750000
2023-08,8.333333
2023-09,0.000000


In [55]:
retention_rate.to_excel("taxa_retencao.xlsx")
churn_rate.to_excel("taxa_churn.xlsx")
risk_churn_rate.to_excel("taxa_risco_churn.xlsx")
revival_rate.to_excel("taxa_reativacao.xlsx")

In [57]:
df

,user_id,purchase_date,first_purchase_date,last_purchase_date,first_purchase_month,purchase_month,is_new_customer,cliente_retido_30,cliente_reativado,days_since_last_purchase,cliente_risco_churn,cliente_churn
0,1,2023-01-01,2023-01-01,2023-12-01,2023-01,2023-01,True,True,False,31,False,False
1,1,2023-02-01,2023-01-01,2023-12-01,2023-01,2023-02,False,True,False,31,False,False
2,1,2023-12-01,2023-01-01,2023-12-01,2023-01,2023-12,False,True,False,31,False,False
3,2,2023-01-01,2023-01-01,2023-12-01,2023-01,2023-01,True,False,True,31,True,False
4,2,2023-06-01,2023-01-01,2023-12-01,2023-01,2023-06,False,False,True,31,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
241,229,2023-10-01,2023-08-01,2023-10-01,2023-08,2023-10,False,True,False,92,True,False
242,229,2023-08-01,2023-08-01,2023-10-01,2023-08,2023-08,True,True,False,92,True,False
243,230,2023-09-01,2023-09-01,2023-10-01,2023-09,2023-09,True,True,False,92,True,False
244,230,2023-10-01,2023-09-01,2023-10-01,2023-09,2023-10,False,True,False,92,True,False


In [59]:
df

,user_id,purchase_date,first_purchase_date,last_purchase_date,first_purchase_month,purchase_month,is_new_customer,cliente_retido_30,cliente_reativado,days_since_last_purchase,cliente_risco_churn,cliente_churn
0,1,2023-01-01,2023-01-01,2023-12-01,2023-01,2023-01,True,True,False,31,False,False
1,1,2023-02-01,2023-01-01,2023-12-01,2023-01,2023-02,False,True,False,31,False,False
2,1,2023-12-01,2023-01-01,2023-12-01,2023-01,2023-12,False,True,False,31,False,False
3,2,2023-01-01,2023-01-01,2023-12-01,2023-01,2023-01,True,False,True,31,True,False
4,2,2023-06-01,2023-01-01,2023-12-01,2023-01,2023-06,False,False,True,31,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
241,229,2023-10-01,2023-08-01,2023-10-01,2023-08,2023-10,False,True,False,92,True,False
242,229,2023-08-01,2023-08-01,2023-10-01,2023-08,2023-08,True,True,False,92,True,False
243,230,2023-09-01,2023-09-01,2023-10-01,2023-09,2023-09,True,True,False,92,True,False
244,230,2023-10-01,2023-09-01,2023-10-01,2023-09,2023-10,False,True,False,92,True,False


In [61]:
df.to_excel("analise_retencao.xlsx")

In [63]:
retention_rate

,user_id
first_purchase_date,
2023-01,75.000000
2023-02,75.000000
2023-03,33.333333
2023-04,62.500000
2023-05,33.333333
2023-06,44.444444
2023-07,25.000000
2023-08,50.000000
2023-09,50.000000


In [141]:
df.to_excel("analise_retencao.xlsx")

In [67]:
retention_rate.to_excel("taxa_retencao.xlsx")